In [1]:
import os
os.chdir('..')

from puzzle_generator import *

In [2]:
width = 6
height = 4
puzzle = RectangularPuzzle(
    width=width,
    height=height
)

puzzle.solving_timeout = 10

puzzle.naming = {
    None: ' ',
    '0': ' ',
    'm': '.',
    'b': '*',
    'h': '-',
    'v': '|',
}

puzzle.latex_naming = {
    None: ' ',
    '0': ' ',
    'm': '\\Huge \\raisebox{-.5em}{\\ensuremath{\\cdot}}',
    'b': '\\Large \\ensuremath{\\bullet}',
    'h': '\\Large \\raisebox{-.5em}{\\textbf{---}}',
    'v': '\\Large \\textbf{|}',
}
for i in range(1, 25):
    puzzle.latex_naming[str(i)] = f'\\Large \\ensuremath{{{i}}}'

puzzle.domain_program += """
    #const min_num_mines=4.
    #const max_num_mines=8.
    
    1 { guessed_number(num_big_mines,1) } 1.
    1 { guessed_number(num_h_mines,1) } 1.
    1 { guessed_number(num_v_mines,1) } 1.
    
    value(m). % regular mines (range 1)
    value(b). % big mines (range 2)
    value(h;v). % horizontal/vertical mines (range 1)
    value(N) :- num(N).
    num(0..24). % 24 is the max number of neighboring mines (regular/big)
"""

puzzle.puzzle_gen_program += """
    { puzzle(C,V) : cell(C), value(V) }.
    :- puzzle(C,m), cell(C).
    :- puzzle(C,b), cell(C).
    :- puzzle(C,h), cell(C).
    :- puzzle(C,v), cell(C).
    :- puzzle(C,0), cell(C).
"""
puzzle.puzzle_constraints_program += """
    :- not min_num_mines { solution(C,m) } max_num_mines.
    
    #minimize { 1,puzzle(C,V) : puzzle(C,V) }.
"""

puzzle.solution_program = """
    1 { solution(C,V) : value(V) } 1 :- cell(C).
    solution(C,V) :- puzzle(C,V).
"""
puzzle.solution_program += enc_library['ring_around_cell']
puzzle.solution_program += """
    attacks(C1,C2) :- cell(C1), cell(C2), C1 != C2,
        solution(C1,m), ring_around_cell(C1,C2,D), D <= 1.
    attacks(C1,C2) :- cell(C1), cell(C2), C1 != C2,
        solution(C1,b), ring_around_cell(C1,C2,D), D > 0, D <= 2.
    attacks(C1,C2) :- cell(C1), cell(C2),
        C1 = c(Row,Col1), C2 = c(Row,Col2), |Col1-Col2| = 1,
        solution(C1,h).
    attacks(C1,C2) :- cell(C1), cell(C2),
        C1 = c(Row1,Col), C2 = c(Row2,Col), |Row1-Row2| = 1,
        solution(C1,v).

    :- cell(C), solution(C,N), num(N),
        N != #count { D : cell(D), attacks(D,C) }.
    
    :- guessed_number(num_big_mines,N),
        not N { solution(C,b) } N.
    :- guessed_number(num_v_mines,N),
        not N { solution(C,v) } N.
    :- guessed_number(num_h_mines,N),
        not N { solution(C,h) } N.
"""

In [3]:
puzzle.generate(
    verbose=True,
    precompute_solution=False,
)

.......
Stopped after solving timeout..
Solving time: 10.90 seconds



In [4]:
print(puzzle.pretty_repr_puzzle())

| |4|2| | |4|
| | |1| | | |
| |3| |2| | |
| |1| | |1| |

num_h_mines = 1
num_v_mines = 1
num_big_mines = 1


In [5]:
print(puzzle.pretty_repr_solution())

|.|4|2|-|.|4|
|.|||1|*|.|.|
|1|3|1|2|3|3|
| |1|1|1|1|1|


In [6]:
print(puzzle.latex_repr_puzzle())


\begin{tikzpicture}[scale=1]
  \begin{scope}
    \draw (0,0) grid (6,4);
    \draw[ultra thick]
      (0,0) rectangle (6,4);
      \node[anchor=center] at (1.5,3.5) {\Large \ensuremath{4}};
      \node[anchor=center] at (2.5,3.5) {\Large \ensuremath{2}};
      \node[anchor=center] at (5.5,3.5) {\Large \ensuremath{4}};
      \node[anchor=center] at (2.5,2.5) {\Large \ensuremath{1}};
      \node[anchor=center] at (1.5,1.5) {\Large \ensuremath{3}};
      \node[anchor=center] at (3.5,1.5) {\Large \ensuremath{2}};
      \node[anchor=center] at (1.5,0.5) {\Large \ensuremath{1}};
      \node[anchor=center] at (4.5,0.5) {\Large \ensuremath{1}};
  \end{scope}
\end{tikzpicture}



In [7]:
print(puzzle.latex_repr_solution())


\begin{tikzpicture}[scale=1]
  \begin{scope}
    \draw (0,0) grid (6,4);
    \draw[ultra thick]
      (0,0) rectangle (6,4);
      \node[anchor=center] at (0.5,3.5) {\Huge \raisebox{-.5em}{\ensuremath{\cdot}}};
      \node[anchor=center] at (1.5,3.5) {\Large \ensuremath{4}};
      \node[anchor=center] at (2.5,3.5) {\Large \ensuremath{2}};
      \node[anchor=center] at (3.5,3.5) {\Large \raisebox{-.5em}{\textbf{---}}};
      \node[anchor=center] at (4.5,3.5) {\Huge \raisebox{-.5em}{\ensuremath{\cdot}}};
      \node[anchor=center] at (5.5,3.5) {\Large \ensuremath{4}};
      \node[anchor=center] at (0.5,2.5) {\Huge \raisebox{-.5em}{\ensuremath{\cdot}}};
      \node[anchor=center] at (1.5,2.5) {\Large \textbf{|}};
      \node[anchor=center] at (2.5,2.5) {\Large \ensuremath{1}};
      \node[anchor=center] at (3.5,2.5) {\Large \ensuremath{\bullet}};
      \node[anchor=center] at (4.5,2.5) {\Huge \raisebox{-.5em}{\ensuremath{\cdot}}};
      \node[anchor=center] at (5.5,2.5) {\Huge \raisebox{